In [1]:
import kagglehub
import pandas as pd

import numpy as np
from pathlib import Path

In [14]:
PATH_RAW: Path = Path('./assets/raw')


In [3]:
# # Download latest version
# path = kagglehub.dataset_download("paultimothymooney/stock-market-data")
#
# print("Path to dataset files:", path)

In [4]:
path = '/home/lipov/.cache/kagglehub/datasets/paultimothymooney/stock-market-data/versions/74'

In [5]:
!ls /home/lipov/.cache/kagglehub/datasets/paultimothymooney/stock-market-data/versions/74/stock_market_data/sp500/csv

AAL.csv    BSHI.csv   ED.csv	 HPQ.csv    MCHP.csv   PH.csv	  TIME.csv
AAP.csv    BSX.csv    EFX.csv	 HRB.csv    MCK.csv    PHM.csv	  TJX.csv
AAPL.csv   BWA.csv    EIX.csv	 HRL.csv    MCO.csv    PKG.csv	  TMO.csv
ABBV.csv   BXP.csv    EL.csv	 HSIC.csv   MDLZ.csv   PKI.csv	  TMUS.csv
ABC.csv    CAG.csv    EMN.csv	 HST.csv    MDT.csv    PLD.csv	  TRAUF.csv
ABMD.csv   CAH.csv    EMR.csv	 HSY.csv    MET.csv    PM.csv	  TROW.csv
ABT.csv    CAT.csv    ENS.csv	 HTLF.csv   MGM.csv    PNR.csv	  TRV.csv
ACN.csv    CB.csv     EOG.csv	 HUM.csv    MHK.csv    PNW.csv	  TSCO.csv
A.csv	   CCI.csv    EQIX.csv	 IBM.csv    MKTX.csv   PNWRF.csv  TSN.csv
ADI.csv    C.csv      EQR.csv	 ICE.csv    MLM.csv    PPG.csv	  TTWO.csv
ADM.csv    CDE.csv    ES.csv	 IDXX.csv   MMC.csv    PRU.csv	  TW.csv
ADP.csv    CDNS.csv   ESS.csv	 IEX.csv    MMM.csv    PSX.csv	  TWTR.csv
ADSK.csv   CF.csv     EW.csv	 IFF.csv    MNST.csv   PVH.csv	  TXN.csv
AEE.csv    CFG.csv    EXR.csv	 ILMN.csv   MO.csv     PWR.csv	  TXT.csv
AEP.c

In [6]:
from pathlib import Path

sp500_path = Path(path + '/stock_market_data/sp500/csv')
sp500_path

PosixPath('/home/lipov/.cache/kagglehub/datasets/paultimothymooney/stock-market-data/versions/74/stock_market_data/sp500/csv')

In [7]:
df_goog = pd.read_csv(sp500_path / 'GOOG.csv', parse_dates=['Date'], index_col='Date', usecols=['Date', 'Adjusted Close'], dayfirst=True)

In [8]:
df_goog.head(3)

,Adjusted Close
Date,
2004-08-19,2.499133
2004-08-20,2.697639
2004-08-23,2.724787


In [9]:
df_goog['Adjusted Close'].pct_change()

Date
2004-08-19         NaN
2004-08-20    0.079430
2004-08-23    0.010064
2004-08-24   -0.041408
2004-08-25    0.010775
                ...   
2022-12-06   -0.025633
2022-12-07   -0.022197
2022-12-08   -0.012612
2022-12-09   -0.009367
2022-12-12   -0.006393
Name: Adjusted Close, Length: 4612, dtype: float64

In [10]:
df_goog['log_return'] = np.log(df_goog["Adjusted Close"] / df_goog["Adjusted Close"].shift(1))
df_goog.head(3)

,Adjusted Close,log_return
Date,,
2004-08-19,2.499133,NaN
2004-08-20,2.697639,0.076433
2004-08-23,2.724787,0.010013


In [11]:
df_goog = df_goog.drop(columns=['Adjusted Close'])

In [12]:
df_goog = df_goog.dropna()
df_goog.head(2)

,log_return
Date,
2004-08-20,0.076433
2004-08-23,0.010013


In [15]:
df_goog.to_parquet(PATH_RAW / 'goog_return.parquet', index=True)

In [16]:
csv_files = list(sp500_path.glob('*.csv'))

In [17]:
all_returns = []

In [18]:
for file in csv_files:
    ticker = file.stem

    df = pd.read_csv(
        file,
        usecols=['Date', 'Adjusted Close'],
        parse_dates=['Date'],
        index_col='Date',
        dayfirst=True
    ).sort_index()

    df[ticker] = np.log(df['Adjusted Close'] / df['Adjusted Close'].shift(1))

    df = df[ticker]

    all_returns.append(df)

/home/lipov/projects/linear_regression/.venv/lib/python3.13/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/lipov/projects/linear_regression/.venv/lib/python3.13/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [19]:
df_returns = pd.concat(all_returns, axis=1).dropna(how='all')
df_returns.shape

(13355, 409)